### Single survey POC

In [4]:
import sys
import os
from reverie import ReverieServer
from tqdm import tqdm
from contextlib import contextmanager

@contextmanager
def suppress_output():
    # Save the current stdout
    original_stdout = sys.stdout
    # Redirect stdout to the null device
    sys.stdout = open(os.devnull, 'w')
    try:
        yield
    finally:
        # Restore the original stdout
        sys.stdout.close()
        sys.stdout = original_stdout

In [2]:
# remove temp folder if it exists
if os.path.exists("../../environment/frontend_server/storage/_temp"):
    os.system("rm -rf ../../environment/frontend_server/storage/_temp")

PATH = "base_the_ville_smol_elections_5_voters_2days/base_the_ville_smol_elections_5_voters_2days-s-142-15463-15594"

rs = ReverieServer(PATH, "_temp")

In [3]:
# with suppress_output():
#     o = rs.personas["Maria Lopez"].open_convo_session("analysis", direct=True, question="Who are you planning to vote in the coming elections?")

In [5]:
survey_question = "Who are you planning to vote in the coming elections?"

convos = []
for persona in tqdm(rs.personas.values()):
    with suppress_output():
        convos.append(persona.open_convo_session("analysis", direct=True, question=survey_question)[-1])


100%|██████████| 7/7 [00:24<00:00,  3.50s/it]


In [6]:
convos

[['Isabella Rodriguez',
  'I am still in the process of evaluating the candidates Klaus Mauller and Maria Lopez for the upcoming town elections. I am planning to discuss my thoughts on this matter with Adam Smith over coffee at Hobbs Cafe tomorrow. I want to make an informed decision and consider the best candidate for our community.'],
 ['Maria Lopez',
  "I'm still undecided at the moment. I'm researching and learning more about the candidates and their policies before making my decision. It's important to me to make an informed choice based on the issues that matter most to me and my community."],
 ['Klaus Mueller',
  "I haven't decided yet. I'm still gathering information on the candidates and their platforms. I think it's important to make an informed decision based on their stance on issues that matter to me, like social justice and community development."],
 ['Abigail Chen',
  "I'm still exploring all the candidates' proposals and attending events to understand their platforms be

In [20]:
import persona.prompt_template.gpt_structure as gpt_structure

gpt_param = {"engine": "gpt-35-turbo-0125", "max_tokens": 250, 
            "temperature": 0, "top_p": 1, "stream": False,
            "frequency_penalty": 0, "presence_penalty": 0, "stop": None}

extract_prompt = "You are given a response to a survey question \"Who are you planning to vote in the coming elections?\". Extract and write the name of the candidate mentioned in the response or write None if the responder have not decided yet. Respond with the name of the candidate or None. Response: {response}"

In [25]:
for convo in convos:
    print(f"Person: {convo[0]}")
    print(f"Response: {convo[-1]}")
    print(f"Predicted: {gpt_structure.GPT_request(extract_prompt.format(response=convo[-1]), gpt_param)}")
    print("\n\n")

Person: Isabella Rodriguez
Response: I am still in the process of evaluating the candidates Klaus Mauller and Maria Lopez for the upcoming town elections. I am planning to discuss my thoughts on this matter with Adam Smith over coffee at Hobbs Cafe tomorrow. I want to make an informed decision and consider the best candidate for our community.
Predicted: None



Person: Maria Lopez
Response: I'm still undecided at the moment. I'm researching and learning more about the candidates and their policies before making my decision. It's important to me to make an informed choice based on the issues that matter most to me and my community.
Predicted: None



Person: Klaus Mueller
Response: I haven't decided yet. I'm still gathering information on the candidates and their platforms. I think it's important to make an informed decision based on their stance on issues that matter to me, like social justice and community development.
Predicted: None



Person: Abigail Chen
Response: I'm still explo

### Multisurvey

In [53]:
import sys
import os
import pickle
import pandas as pd
from tqdm import tqdm
from contextlib import contextmanager

import persona.prompt_template.gpt_structure as gpt_structure
from reverie import ReverieServer


@contextmanager
def suppress_output():
    # Save the current stdout
    original_stdout = sys.stdout
    # Redirect stdout to the null device
    sys.stdout = open(os.devnull, 'w')
    try:
        yield
    finally:
        # Restore the original stdout
        sys.stdout.close()
        sys.stdout = original_stdout

def get_folders(path):
    return [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]

def clean_start_server(path):
    if os.path.exists("../../environment/frontend_server/storage/_temp"):
        os.system("rm -rf ../../environment/frontend_server/storage/_temp")

    rs = ReverieServer(path, "_temp")
    return rs

def get_survey_responses(rs, survey_question):
    convos = []
    for persona in tqdm(rs.personas.values()):
        with suppress_output():
            convos.append(persona.open_convo_session("analysis", direct=True, question=survey_question)[-1])
    return convos

def extract_survey_responses(convos, extract_prompt, gpt_param):
    predictions = []
    for convo in convos:
        predictions.append((convo[-1], gpt_structure.GPT_request(extract_prompt.format(response=convo[-1]), gpt_param)))
    return predictions

def update_results(results, predictions):
    for i, (response, prediction) in enumerate(predictions):
        if prediction in results:
            results[prediction] += 1
        else:
            results[prediction] = 1
    return results

def save_file(results, path):
    with open(path, "wb") as f:
        pickle.dump(results, f)

def load_file(path):
    with open(path, "rb") as f:
        return pickle.load(f)

gpt_param = {"engine": "gpt-35-turbo-0125", "max_tokens": 250, 
            "temperature": 0, "top_p": 1, "stream": False,
            "frequency_penalty": 0, "presence_penalty": 0, "stop": None}

extract_survey_prompt = "You are given a response to a survey question \"Who are you planning to vote in the coming elections?\". Extract and write the name of the candidate mentioned in the response or write None if the responder have not decided yet. Respond with the name of the candidate or None. Response: {response}"

In [54]:
PATH_SIM = "base_the_ville_smol_elections_5_voters_2days"

folders = get_folders(f"../../environment/frontend_server/storage/{PATH_SIM}")
folders_sorted = sorted(folders, key=lambda x: int(x.split("-")[-3]))

results = []
all_convos = []
all_predictions = []
df_survey = pd.DataFrame(columns=list(map(lambda x: x.split("-")[-3], folders_sorted)))
for PATH_STAGE in tqdm(folders_sorted):
    rs = clean_start_server(f"{PATH_SIM}/{PATH_STAGE}")
    convos = get_survey_responses(rs, "Who are you planning to vote in the coming elections?")
    predictions = extract_survey_responses(convos, extract_survey_prompt, gpt_param)
    
    results.append(update_results({}, predictions))
    all_convos.append(convos)
    all_predictions.append(predictions)

    save_file(results[-1], f"results.pkl")
    save_file(all_convos[-1], f"convos.pkl")
    save_file(all_predictions[-1], f"predictions.pkl")
    print(results[-1])

  1%|          | 1/151 [00:19<49:11, 19.68s/it]

{'None': 7}


  1%|▏         | 2/151 [00:40<50:49, 20.47s/it]

{'None': 7}


  2%|▏         | 3/151 [01:01<51:14, 20.77s/it]

...

{'None': 7}


 15%|█▌        | 23/151 [08:57<50:33, 23.70s/it]

{'None': 7}


 16%|█▌        | 24/151 [09:21<50:09, 23.69s/it]

{'None': 7}
